In [1]:
print("Lets Start")

Lets Start


In [2]:
import os
print(os.getcwd())


c:\Users\hp\Medical_Chatbot\research


In [4]:
import os
os.chdir("../")


In [5]:
import os
print(os.getcwd())


c:\Users\hp


In [6]:
import os
os.chdir(r"C:\Users\hp\Medical_Chatbot")
print("Current Directory:", os.getcwd())


Current Directory: C:\Users\hp\Medical_Chatbot


In [7]:
import os
print(os.getcwd())


C:\Users\hp\Medical_Chatbot


In [8]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [9]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [14]:
extracted_data = load_pdf_file(data="C:/Users/hp/Medical_Chatbot/Data/")


In [15]:
extracted_data

[Document(metadata={'source': 'C:\\Users\\hp\\Medical_Chatbot\\Data\\forchatbot.pdf', 'page': 0}, page_content='Investing in\nMENTAL HEALTH\n'),
 Document(metadata={'source': 'C:\\Users\\hp\\Medical_Chatbot\\Data\\forchatbot.pdf', 'page': 1}, page_content='This publication was produced by the Department of Mental Health and Substance Dependence, \nNoncommunicable Diseases and Mental Health, World Health Organization, Geneva.\nFor further information and feedback, please contact: \nDepartment of Mental Health and Substance Dependence, Avenue Appia 20, 1211 Geneva 27, Switzerland\nTel: +41 22 791 21 11, fax: +41 22 791 41 60, e-mail: mnh@who.int, website: www.who.int/mental_health\nWHO Library Cataloguing-in-Publication Data\nWorld Health Organization.\nInvesting in mental health.\n1.Mental disorders - economics  2.Mental disorders - therapy  3.Mental health services - economics  \n4.Mental health services - economics  5.Cost of illness  6.Investments  I.Title.\nISBN 92 4 156257 9 (NLM c

In [16]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks


In [17]:

text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1119


In [18]:
from langchain.embeddings import HuggingFaceEmbeddings

In [19]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [20]:
embeddings = download_hugging_face_embeddings()

C:\Users\hp\AppData\Local\Temp\ipykernel_11236\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\ProgramData\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
query_result= embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [23]:
query_result

[-0.03447731211781502,
 0.03102317824959755,
 0.006734973285347223,
 0.026108987629413605,
 -0.03936202451586723,
 -0.16030248999595642,
 0.06692396104335785,
 -0.006441520527005196,
 -0.047450557351112366,
 0.014758833684027195,
 0.0708753690123558,
 0.05552764981985092,
 0.019193340092897415,
 -0.026251304894685745,
 -0.010109521448612213,
 -0.026940487325191498,
 0.022307438775897026,
 -0.022226588800549507,
 -0.1496925950050354,
 -0.017493003979325294,
 0.007676261477172375,
 0.054352227598428726,
 0.003254449460655451,
 0.031725913286209106,
 -0.0846213772892952,
 -0.029406029731035233,
 0.05159557983279228,
 0.0481240414083004,
 -0.0033148375805467367,
 -0.05827924236655235,
 0.041969284415245056,
 0.02221062034368515,
 0.1281888782978058,
 -0.0223389845341444,
 -0.01165628433227539,
 0.06292835623025894,
 -0.03287626802921295,
 -0.09122604876756668,
 -0.031175415962934494,
 0.052699532359838486,
 0.047034814953804016,
 -0.08420310914516449,
 -0.030056163668632507,
 -0.0207448359

In [24]:
from dotenv import load_dotenv
load_dotenv()


True

In [26]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "mentalhealth-bot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
)

In [27]:


import os

# # Directly assign the API keys here
# Check if keys are set
if not PINECONE_API_KEY or not OPENAI_API_KEY:
    raise ValueError("API keys are missing or incorrect!")

# Set the environment variables for the application to use
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("API keys are set successfully.")




API keys are set successfully.


In [28]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [29]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [30]:
docsearch

In [31]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [32]:
retriever_docs = retriever.invoke("What is mental health")

In [33]:
retriever_docs

[Document(id='3537f899-363b-4971-be73-a8058f76a398', metadata={'page': 3.0, 'source': 'C:\\Users\\hp\\Medical_Chatbot\\Data\\mind-mental-and-physical-activity-toolkit-guide-1.pdf'}, page_content='What is mental health? \nIn many ways, mental health is just like physical health: everybody has it and we need to take care of it. \nMental health is a continuum, ranging from good to poor.\nGood mental health means being generally able to think, feel and \nreact in the ways that you need and want to live your life. But if you \ngo through a period of poor mental health you might find the ways \nyou’re frequently thinking, feeling or reacting become difficult, or even'),
 Document(id='3e9e0815-b5ac-4061-b348-d317a78b8789', metadata={'page': 8.0, 'source': 'C:\\Users\\hp\\Medical_Chatbot\\Data\\forchatbot.pdf'}, page_content='7\nMental health is more than the mere lack\nof mental disorders. The positive dimen-\nsion of mental health is stressed in WHO’s\ndeﬁnition of health as contained in its

In [34]:
import openai

# Set the API key
openai.api_key = "OPENAI_API_KEY"

# Now try the OpenAI setup from LangChain
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)


In [35]:

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are a mental health assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)



prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [36]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [37]:
response = rag_chain.invoke({"input": "what is Mental health?"})
print(response["answer"])



Mental health is a state of well-being that includes our emotional, psychological, and social well-being. It is not just the absence of mental disorders, but also the ability to cope with stress, function well in daily life, and contribute to our communities. It is important to take care of our mental health just like we do our physical health. 


In [38]:
import os
print(PINECONE_API_KEY)  # Check if the value is None

pcsk_w6r6a_KCytbnnV4YyXer1otzUKhHSUR5EbY2257Cv6xV6skQ3LYLckm5GvC3e6uSix6td
